In [1]:
%run 'pymol_and_pdb_functions.py'
start_pymol()

In [2]:
import itertools

%run 'ks01_Functions_only.ipynb'
notebook_prefix = 'ks16'
image_counter = Counter()

/Users/karen/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/karen/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### Reading 24-mer structure

In [3]:
his3_aligned_to_4lom_assembly_file = os.path.join(structure_predictions_folder, 'his3_24mer_assembly', 
                                                  'his3_swiss_aligned_to_4lom_assembly.pdb')
structure = Bio.PDB.PDBParser().get_structure('his3_swiss_assembly', his3_aligned_to_4lom_assembly_file)
model = structure[0]

/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 37657.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 37675.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 37693.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 37711.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 37729.
  PDBConstructionWarning)
/Users/karen/anaconda/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:85: PDBConstructi

### Calculating distances

In [4]:
distance_dict = OrderedDict()
prefix = 'res_'

constant_chain = model['A']
positions_in_crystal = [r.id[1] for r in constant_chain.get_residues() if r.get_resname() in aa3]

f = FloatProgress(min=0, max=len(list(constant_chain.get_residues())))
display(f)

for residue in constant_chain.get_residues():
    if residue.get_resname() in aa3:
        distance_dict[prefix + '%s' %residue.id[1]] = OrderedDict()

        # distances to other residues
        for position in positions_in_crystal:
            distances = []
            for other_chain in model.get_chains():
                distances.append(get_distance_between_residues(residue, other_chain[position]))
            distance_dict[prefix + '%s' %residue.id[1]][prefix + '%s' %position] = min(distances)

        # distances to Mn ions
        for position in [302, 303, 304]:
            distances = []
            for other_chain in model.get_chains():
                other_residue = [r for r in other_chain if position == r.id[1]][0]
                distances.append(get_distance_between_residues(residue, other_residue))
            distance_dict[prefix + '%s' %residue.id[1]]['Mn_' + '%s' %position] = min(distances)
        distance_dict[prefix + '%s' %residue.id[1]]['Mn_substrate_bound'] = min(
            distance_dict[prefix + '%s' %residue.id[1]]['Mn_' + '302'],
            distance_dict[prefix + '%s' %residue.id[1]]['Mn_' + '303'])
            
        # distances to substrate
        for position in [301]:
            distances = []
            for other_chain in model.get_chains():
                other_residue = [r for r in other_chain if position == r.id[1]][0]
                distances.append(get_distance_between_residues(residue, other_residue))
            distance_dict[prefix + '%s' %residue.id[1]]['substrate'] = min(distances)
    f.value += 1

In [5]:
structural_data = pd.DataFrame.from_dict(distance_dict, orient='index')
new_index = sorted(structural_data.index.values, key=lambda s: int(s[4:]))
structural_data = structural_data.reindex(new_index)
structural_data['position'] = structural_data.index.map(lambda s: int(s[4:]))

### Secondary structure

In [21]:
# information from PyMol? using psico?
# import psico
# import psico.fullinit
# import psico.helping
# psico.helping.write_html_ref('psico-commands.html')

helices = range (60,74) + range (89,109) + range (157,172) + range (185,205)
sheets = range(5,12) + range(15,23) + range(52,55) + range(76,84) + range(117,125) + range(127,135) + range(138,145) + range(173,182)
disordered = [position for position in range(1, len(Scer_Uniprot)) if position not in helices and position not in sheets]

def get_secondary_structure(position):
#     print position, position in helices, position in sheets
    assert ~(position in helices and position in sheets)
    if position in helices:
        return 'helix'
    elif position in sheets:
        return 'sheet'
    else:
        return 'disordered'

structural_data['secondary_structure'] = structural_data['position'].apply(get_secondary_structure)

### Finding interface residues

In [ ]:
ascii_letters_upper = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

cmd.reinitialize()
cmd.load(his3_aligned_to_4lom_assembly_file, 'his3_swiss_assembly')

interfaces = {}
f = FloatProgress(min=0, max=len(list(itertools.combinations(range(24), 2))))
display(f)
for chain_index1, chain_index2 in (itertools.combinations(range(24), 2)):
    chain1, chain2 = ascii_letters_upper[chain_index1], ascii_letters_upper[chain_index2]
    returned = interfaceResidues('his3_swiss_assembly', cA = 'c. %s' %chain1, cB = 'c. %s' %chain2)
    interfaces[chain1, chain2] = returned
    f.value += 1
    
interface_residues = []
for k,v in interfaces.items():
    interface_residues.extend([e[1] for e in v])
interface_residues = sorted(list(set([int(p) for p in interface_residues])))

structural_data['interface'] = structural_data.index.map(lambda s: int(s[4:]) in interface_residues)

### Reading data on sign epistasis from Lucas

In [7]:
# Relative >>> absolute position
position_translation = pd.read_table(files_dump_folder + 'position_translation.csv')
position_translation.set_index('relative_position', inplace=True)

def get_absolute_position(segment_number, relative_position):
    return int(position_translation.iloc[relative_position]['S'+str(segment_number)])

In [8]:
lucas_sign_epistasis = pd.read_csv(files_dump_folder + 'sign_epistasis/' + 'lucas_sign_epistasis.csv')
lucas_reciprocal_sign_epistasis = pd.read_csv(files_dump_folder + 'sign_epistasis/' + 'lucas_reciprocal_sign_epistasis.csv')

p_value_threshold = 0.01
filtered = lucas_sign_epistasis[lucas_sign_epistasis['pBon'] < p_value_threshold]
sign_epistasis_positions = set.union(set(filtered['VarPos_absolute'].values), 
                                                set(filtered['SubPos_absolute'].values))
sign_epistasis_positions = sorted([int(s) for s in sign_epistasis_positions])
print len(sign_epistasis_positions), 'positions under sign epistasis'

reciprocal_sign_epistasis_positions = set.union(set(lucas_reciprocal_sign_epistasis['position1'].values), 
                                                set(lucas_reciprocal_sign_epistasis['position1'].values))
reciprocal_sign_epistasis_positions = sorted([int(s) for s in reciprocal_sign_epistasis_positions])
print len(reciprocal_sign_epistasis_positions), 'positions under reciprocal sign epistasis'

structural_data['lucas_sign_epistasis'] = structural_data.index.map(lambda s: int(s[4:]) in sign_epistasis_positions)
structural_data['lucas_reciprocal_sign_epistasis'] = structural_data.index.map(lambda s: int(s[4:]) in reciprocal_sign_epistasis_positions)

94 positions under sign epistasis
59 positions under reciprocal sign epistasis


### Conservation score

In [42]:
Scer_Uniprot = open(os.path.join(files_dump_folder, 'HIS3_saccharomyces_cerevisiae_from_Uniprot_P06633.txt')).read().rstrip()

# numbering starts from zero
conservation_scores = pd.read_table(files_dump_folder + 'conservation_score.csv')
conservation_scores['positions_Uniprot_P06633'] = conservation_scores.position_in_alignment.apply(
    lambda p: get_wt_position(p))

structural_data['segment'] = structural_data['position'].apply(lambda p: conservation_scores.iloc[p-1]['segment'])
structural_data['Scer_aa'] = structural_data['position'].apply(lambda p: Scer_Uniprot[p-1])

structural_data['alignment_entropy'] = structural_data['position'].apply(lambda p: conservation_scores.iloc[p-1]['entropy'])
structural_data['alignment_gap_fraction'] = structural_data['position'].apply(lambda p: conservation_scores.iloc[p-1]['gap_fr'])
structural_data['conservation_score'] = structural_data['position'].apply(lambda p: conservation_scores.iloc[p-1]['score'])

,res_2,res_3,res_4,res_5,res_6,res_7,res_8,res_9,res_10,res_11,...,position,interface,lucas_sign_epistasis,lucas_reciprocal_sign_epistasis,secondary_structure,alignment_entropy,alignment_gap_fraction,conservation_score,segment,Scer_aa
res_2,0.000000,1.344903,3.929120,6.801780,10.320789,12.860404,16.926117,20.202641,23.694307,24.654099,...,2,False,False,False,disordered,0.819458,0.504718,0.089419,NaN,T
res_3,1.344903,0.000000,1.335484,2.674407,7.090796,9.983333,13.894839,17.330608,20.780022,24.223778,...,3,False,False,False,disordered,0.825919,0.811640,0.032790,NaN,E
res_4,3.929120,1.335484,0.000000,1.344155,4.498135,8.109940,11.214881,14.755608,17.634836,21.506519,...,4,False,False,False,disordered,0.060761,0.057111,0.885598,NaN,Q
res_5,6.801780,2.674407,1.344155,0.000000,1.344352,4.719050,8.127696,11.605848,15.022046,18.489553,...,5,False,False,False,sheet,0.781385,0.030615,0.211922,NaN,K
res_6,10.320789,7.090796,4.498135,1.344352,0.000000,1.345744,4.493712,8.010320,11.393576,14.883163,...,6,False,False,False,sheet,0.436700,0.011247,0.556965,12.0,A
res_7,12.860404,9.983333,8.109940,4.719050,1.345744,0.000000,1.339997,4.493738,7.964891,11.392092,...,7,False,True,False,sheet,0.787952,0.010532,0.209815,12.0,L
res_8,16.926117,13.894839,11.214881,8.127696,4.493712,1.339997,0.000000,1.346331,4.322484,7.995336,...,8,False,True,False,sheet,0.523980,0.005781,0.473268,12.0,V
res_9,20.202641,17.330608,14.755608,11.605848,8.010320,4.493738,1.346331,0.000000,1.340466,4.455739,...,9,False,True,True,sheet,0.743294,0.001839,0.256234,12.0,K
res_10,23.694307,20.780022,17.634836,15.022046,11.393576,7.964891,4.322484,1.340466,0.000000,1.344090,...,10,False,False,False,sheet,0.025936,0.000775,0.973309,12.0,R
res_11,24.654099,24.223778,21.506519,18.489553,14.883163,11.392092,7.995336,4.455739,1.344090,0.000000,...,11,False,True,True,sheet,0.754881,0.000000,0.245119,12.0,I


### Saving files

In [22]:
structural_data.to_hdf(files_dump_folder + 'structural_data_for_predicted_24mer.hdf', 'data')
structural_data.reset_index().to_csv(files_dump_folder + 'structural_data_for_predicted_24mer.csv', index=False)